In [1]:
import pandas as pd 
import numpy as np

In [2]:
df=pd.read_csv(r"C:\Data Analyst Project\bank dataset.csv")
df

,Account No,DATE,TRANSACTION DETAILS,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT
0,4.090000e+11,2017-06-29,TRF FROM Indiaforensic SERVICES,2017-06-29,0,1000000,1000000
1,4.090000e+11,2017-07-05,TRF FROM Indiaforensic SERVICES,2017-07-05,0,1000000,2000000
2,4.090000e+11,2017-07-18,FDRL/INTERNAL FUND TRANSFE,2017-07-18,0,500000,2500000
3,4.090000e+11,2017-08-01,TRF FRM Indiaforensic SERVICES,2017-08-01,0,3000000,5500000
4,4.090000e+11,2017-08-16,FDRL/INTERNAL FUND TRANSFE,2017-08-16,0,500000,6000000
...,...,...,...,...,...,...,...
113697,4.090000e+11,2019-03-05,TRF TO 1196428 Indiaforensic SE,2019-03-05,117934,0,-1901902093
113698,4.090000e+11,2019-03-05,FDRL/INTERNAL FUND TRANSFE,2019-03-05,0,300000,-1901602093
113699,4.090000e+11,2019-03-05,FDRL/INTERNAL FUND TRANSFE,2019-03-05,0,300000,-1901302093
113700,4.090000e+11,2019-03-05,IMPS 05-03-20194C,2019-03-05,109869,0,-1901411961


In [3]:
import mysql.connector  

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='2580',  
    database='bank'
)

cursor = conn.cursor()


In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS bank(
    Account_No VARCHAR(50),
    TRANS_DATE DATE,
    TRANSACTION_DETAILS VARCHAR(50),
    VALUE_DATE DATE,
    WITHDRAWAL_AMT INT,
    DEPOSIT_AMT INT,
    BALANCE_AMT INT
)
""")




In [5]:
print(df.columns.tolist())


['Account No', 'DATE', 'TRANSACTION DETAILS', 'VALUE DATE', 'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT']


In [6]:
df.columns = df.columns.str.strip()  # removes leading/trailing spaces
df.columns = df.columns.str.replace(' ', '_')  # optional: replace spaces with underscore
df.columns = df.columns.str.upper()  # optional: make all uppercase to match MySQL
print(df.columns.tolist())


['ACCOUNT_NO', 'DATE', 'TRANSACTION_DETAILS', 'VALUE_DATE', 'WITHDRAWAL_AMT', 'DEPOSIT_AMT', 'BALANCE_AMT']


In [7]:
df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True).dt.date
df['VALUE_DATE'] = pd.to_datetime(df['VALUE_DATE'], dayfirst=True).dt.date


C:\Users\sidha\AppData\Local\Temp\ipykernel_7672\419022649.py:1: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True).dt.date
C:\Users\sidha\AppData\Local\Temp\ipykernel_7672\419022649.py:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['VALUE_DATE'] = pd.to_datetime(df['VALUE_DATE'], dayfirst=True).dt.date


In [8]:
df['TRANS_DATE'] = pd.to_datetime(df['DATE'], dayfirst=True).dt.date

for i, row in df.iterrows():
    sql = """
    INSERT INTO bank (Account_No, TRANS_DATE, TRANSACTION_DETAILS, VALUE_DATE, WITHDRAWAL_AMT, DEPOSIT_AMT, BALANCE_AMT)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, (
        row['ACCOUNT_NO'],
        row['TRANS_DATE'],
        row['TRANSACTION_DETAILS'],
        row['VALUE_DATE'],
        row['WITHDRAWAL_AMT'],
        row['DEPOSIT_AMT'],
        row['BALANCE_AMT']
    ))

conn.commit()
conn.close()
